In [1]:
from PIL import Image
import numpy as np
import os

In [2]:
path='/home/tongwang/data/test_food_image_2/'
im = Image.open('/home/tongwang/data/test_food_image_2/101&Fast_Food$12.bmp')
im_array = np.array(im)
print(im_array.shape)

(500, 500, 3)


In [3]:
rows=500
cols=500
channels=3
train = 120
test = 27

In [4]:
pic_names = os.listdir(path)
pics = []
labels = []
for name in pic_names:
    im = Image.open(path+name)
    im_array = np.array(im)
    if len(im_array.shape) == 3:
        pics.append(im_array)
        label = name[name.index('&')+1:name.index('$')]
        if label == 'Fast_Food':
            labels.append(0)
        elif label == 'Pizza':
            labels.append(1)
        else:
            labels.append(2)
        

X = np.zeros((len(pics),channels,rows,cols))
for i in range(len(pics)):
    for j in range(channels):
        X[i][j] = pics[i][:,:,j]
y = np.array(labels)

print(X.shape)
print(y.shape)

np.random.seed(10)
np.random.shuffle(X)
np.random.seed(10)
np.random.shuffle(y)
X_train = X[:train]
y_train = y[:train]
X_test = X[train:]
y_test = y[train:]

(147, 3, 500, 500)
(147,)


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using Theano backend.
Using gpu device 0: GeForce GTX 980 Ti (CNMeM is enabled with initial size: 80.0% of memory, cuDNN not available)


In [17]:
batch_size = 32
nb_classes = 3
nb_epoch = 20

In [18]:
Y_train = np_utils.to_categorical(y_train, 3)
Y_test = np_utils.to_categorical(y_test, 3)

In [23]:
model = Sequential()
model.add(Convolution2D(4, 50, 50,border_mode='valid', subsample=(10,10),
                        input_shape=(channels, rows, cols)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(4, 3, 3,border_mode='valid', subsample=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 120 samples, validate on 27 samples
Epoch 1/20
120/120 [==============================] - 1s - loss: 8.7683 - acc: 0.4083 - val_loss: 9.0355 - val_acc: 0.2963
Epoch 2/20
120/120 [==============================] - 1s - loss: 9.2149 - acc: 0.3750 - val_loss: 9.4317 - val_acc: 0.3704
Epoch 3/20
120/120 [==============================] - 1s - loss: 10.2672 - acc: 0.3417 - val_loss: 9.3219 - val_acc: 0.3704
Epoch 4/20
120/120 [==============================] - 1s - loss: 7.9562 - acc: 0.4333 - val_loss: 4.3516 - val_acc: 0.1852
Epoch 5/20
120/120 [==============================] - 1s - loss: 7.4640 - acc: 0.3750 - val_loss: 9.5515 - val_acc: 0.4074
Epoch 6/20
120/120 [==============================] - 1s - loss: 10.9658 - acc: 0.3167 - val_loss: 8.9545 - val_acc: 0.4444
Epoch 7/20
120/120 [==============================] - 1s - loss: 11.1467 - acc: 0.3000 - val_loss: 8.9545 - val_acc: 0.4444
Epoch 8/20
120/120 [==============================] - 1s - loss: 11.8291 - acc: 0.2667 - va

In [24]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_21 (Convolution2D) (None, 4, 46, 46)     30004       convolution2d_input_11[0][0]     
____________________________________________________________________________________________________
activation_41 (Activation)       (None, 4, 46, 46)     0           convolution2d_21[0][0]           
____________________________________________________________________________________________________
maxpooling2d_21 (MaxPooling2D)   (None, 4, 23, 23)     0           activation_41[0][0]              
____________________________________________________________________________________________________
convolution2d_22 (Convolution2D) (None, 4, 21, 21)     148         maxpooling2d_21[0][0]            
___________________________________________________________________________________________

In [17]:
import cv2
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = 4.
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 64
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)
im = cv2.imread('/home/tongwang/data/test_food_image_2/101&Fast_Food$12.bmp')
h = hog.compute(im)
print(h)

[[ 0.46322173]
 [ 0.        ]
 [ 0.        ]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


In [71]:
pics2 = []
hog = cv2.HOGDescriptor()
for name in pic_names:
    im = cv2.imread(path+name)
    h = hog.compute(im)
    if len(im_array.shape) == 3:
        pics2.append(h)

In [73]:
X = np.array(pics2)